FIRST ATTEMPT: STRAIGHT FROM TUTORIAL
This one runs but it doesn't actually stop and ask for human input.

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from langchain.memory import CassandraChatMessageHistory, ConversationBufferMemory
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import json

cloud_config= {
    'secure_connect_bundle': 'secure-connect-choose-your-own-adventure.zip'
}

with open("choose_your_own_adventure_token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]
ASTRA_DB_KEYSPACE = "default_keyspace"
OPENAI_API_KEY = "add_your_own"

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

message_history = CassandraChatMessageHistory(
    session_id = "test1",
    session=session,
    keyspace=ASTRA_DB_KEYSPACE,
    ttl_seconds=3600
)

message_history.clear()

cass_buff_memory = ConversationBufferMemory(
    memory_key = "chat_history",
    chat_memory = message_history
)

template = """
You are now the guide of a mystical journey in the Whispering Woods. 
A traveler named Elara seeks the lost Gem of Serenity. 
You must navigate her through challenges, choices, and consequences, 
dynamically adapting the tale based on the traveler's decisions. 
Your goal is to create a branching narrative experience where each choice 
leads to a new path, ultimately determining Elara's fate. 

Here are some rules to follow:
1. Start by asking the player to choose some kind of weapons that will be used later in the game
2. Have a few paths that lead to success
3. Have some paths that lead to death. If the user dies generate a response that explains the death and ends in the text: "The End.", I will search for this text to end the game

Here is the chat history, use this to understand what to say next: {chat_history}
Human: {human_input}
AI:"""

prompt=PromptTemplate(
    input_variables = ["chat_history", "human_input"],
    template=template
)

llm = OpenAI(openai_api_key=OPENAI_API_KEY)
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=cass_buff_memory
)

choice = "start"

while True:
    response = llm_chain.predict(human_input=choice)
    print(response.strip())

    if "The End." or "The end." in response:
        break

    choice = input("Your reply: ")

SECOND ATTEMPT: ADJUSTING CODE FOR WORKING LANGCHAIN AGENT
Need to make it an agent rather than an agent chain, and add a stop sequence.
This one runs as well, but it only acknowledges the need to get human input and does not give the user the chance to input anything.

In [ ]:
import os
os.environ['ANYSCALE_API_KEY']='add_your_own'

from langchain_community.chat_models import ChatAnyscale
from langchain.agents import AgentExecutor, AgentType, load_tools, initialize_agent, get_all_tool_names, ZeroShotAgent, Tool, ConversationalAgent
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

import random

In [ ]:
cloud_config= {
    'secure_connect_bundle': 'secure-connect-choose-your-own-adventure.zip'
}

with open("choose_your_own_adventure_token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]
ASTRA_DB_KEYSPACE = "default_keyspace"
OPENAI_API_KEY = "add_your_own"

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

message_history = CassandraChatMessageHistory(
    session_id = "test1",
    session=session,
    keyspace=ASTRA_DB_KEYSPACE,
    ttl_seconds=3600
)

message_history.clear()

cass_buff_memory = ConversationBufferMemory(
    memory_key = "chat_history",
    chat_memory = message_history
)

# Creating random dice tool
def roll_dice():
    return random.randrange(1, 20)

dice_tool = Tool.from_function(
    func=roll_dice,
    name="Roll Dice",
    description="Use when you need to roll a dice between 1 and 20"
)

tools = [dice_tool]

prefix = """
You are now the guide of a mystical journey in the Whispering Woods. 
A traveler named Elara seeks the lost Gem of Serenity. 
You must navigate her through challenges, choices, and consequences, 
dynamically adapting the tale based on the traveler's decisions. 
Your goal is to create a branching narrative experience where each choice 
leads to a new path, ultimately determining Elara's fate. 

Here are some rules to follow:
1. Start by asking the player to choose some kind of weapons that will be used later in the game
2. Have a few paths that lead to success
3. Have some paths that lead to death. If the user dies generate a response that explains the death and ends in the text: "The End.", I will search for this text to end the game"""

suffix = """Here is the chat history, use this to understand what to say next: {chat_history}
Human: {human_input}
AI: {agent_scratchpad}"""

prompt=ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=['input', 'chat_history', 'agent_scratchpad']
)

llm_chain = LLMChain(
                llm = ChatAnyscale(anyscale_api_base="https://api.endpoints.anyscale.com/v1",
                    anyscale_api_key=os.getenv("ANYSCALE_API_KEY"),
                    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
                    temperature=0.7,
                    verbose=True),
                    prompt=prompt)
agent=ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=cass_buff_memory, stop=["Human:"]
)

agent_chain.run("start")

THIRD ATTEMPT: USING A CONVERSATIONAL RATHER THAN ZEROSHOT AGENT, ADDING IN TOOL FOR GETTING USER INPUT
This one will allow me to give one or two inputs to questions, but never more than that, and eventually just finishes the story itself. 

In [ ]:
cloud_config= {
    'secure_connect_bundle': 'secure-connect-choose-your-own-adventure.zip'
}

with open("choose_your_own_adventure_token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]
ASTRA_DB_KEYSPACE = "default_keyspace"
OPENAI_API_KEY = "add_your_own"

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

message_history = CassandraChatMessageHistory(
    session_id = "test3",
    session=session,
    keyspace=ASTRA_DB_KEYSPACE,
    ttl_seconds=3600
)

message_history.clear()

cass_buff_memory = ConversationBufferMemory(
    memory_key = "chat_history",
    chat_memory = message_history
)

# Creating random dice tool
def roll_dice():
    return random.randrange(1, 20)

dice_tool = Tool.from_function(
    func=roll_dice,
    name="Roll Dice",
    description="Use when the user asks you to roll a random number between 1 and 20"
)

# human = load_tools(["human"])
def get_input(msg) -> str:
    print(f"AI Message: {msg}. Type your choice:")
    choice = input()
    return choice

human = Tool.from_function(
    func=get_input,
    name="Human Input",
    description="Use when you need to ask the user for input"
)

tools = [dice_tool, human]

prefix = """
You are now the guide of a mystical journey in the Whispering Woods. 
A traveler named Elara seeks the lost Gem of Serenity. 
You must navigate her through challenges, choices, and consequences, 
dynamically adapting the tale based on the traveler's decisions. 
Your goal is to create a branching narrative experience where each choice 
leads to a new path, ultimately determining Elara's fate. 

Here are some rules to follow:
1. Start by asking the player to choose some kind of weapons that will be used later in the game
2. Have a few paths that lead to success
3. Have some paths that lead to death. If the user dies generate a response that explains the death and ends in the text: "The End.", I will search for this text to end the game
4. Always stop and ask for player input when you see "Human:"
"""

suffix = """Here is the chat history, use this to understand what to say next: {chat_history}
Human: {human_input}
AI: {agent_scratchpad}"""

prompt=ConversationalAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=['input', 'chat_history', 'agent_scratchpad']
)

llm_chain = LLMChain(
                llm = ChatAnyscale(anyscale_api_base="https://api.endpoints.anyscale.com/v1",
                    anyscale_api_key=os.getenv("ANYSCALE_API_KEY"),
                    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
                    temperature=0.7,
                    verbose=True),
                    prompt=prompt)

agent=ConversationalAgent(llm_chain=llm_chain, tools=tools, verbose=True)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=cass_buff_memory, stop=["Human:"], handle_parsing_errors=True
)

choice = "start"
while True: 
    response = agent_chain.invoke({"human_input": choice})
    print(response)

    if "The End." or "The end." in response:
        break

    choice = get_input(response)

FOURTH ATTEMPT: USING CHATOPENAI FROM ORIGINAL TUTORIAL BASE
This one provided better input than the original OpenAI chain, but it still stopped rather than asking for input from the user, like so: 

"""Welcome, traveler, to the enchanting Whispering Woods. I am your guide on this mystical journey. Before we begin, I must ask you to choose a weapon to aid you in your quest for the lost Gem of Serenity. You have three options:

1. Bow and arrows: A ranged weapon perfect for precise attacks from a distance.
2. Sword and shield: A classic combination for close combat, offering both offense and defense.
3. Staff: A magical weapon that harnesses the power of the elements to cast spells and defend yourself.

Please choose your weapon, brave Elara.""" <- STOPPED HERE

In [ ]:
# ONLY CHANGE FROM FIRST ATTEMPT: 
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

FUTURE GOALS/ATTEMPTS

I am hoping to get a working ChatOpenAI agent working eventually. I will keep tweaking things to try to get this to work because it feels so close at this point. The ChatOpenAI agent will be far better than the ConversationalAgent from Anyscale because it doesn't follow the Thought-Action-Observation and just responds to questions instead, making the output much closer to the game format I was hoping for. 

Then, I will be able to try to get the stop sequence to work and hopefully it performs better with the OpenAI model than the Mixtral model, but we will see. 